In [1]:
import pandas as pd
import csv
from pprint import pprint
import glob
import os
import sys

In [2]:
# File paths
PLAYER_STATS_FOLDER = 'stats/player/'
TEAM_STATS_FOLDER = 'stats/team/'

In [3]:
def transformEAV(raw, index, values="val", columns="group_stat", aggfunc="last", output=None):
    relational_data = raw.pivot_table(
        index=index,
        columns=columns,
        values=values,
        aggfunc=aggfunc
    ).reset_index()
    if output:
        relational_data.to_csv(f'{output}',index=False)
    return relational_data
        

In [13]:
#rel_team_stats = pd.DataFrame()
csv_files = glob.glob(os.path.join(TEAM_STATS_FOLDER,'season/*_season.csv'))
for file in csv_files:
    filename = os.path.basename(file)
    print(filename)
    team_stats = pd.read_csv(file)
    date = filename[:4]
    team_stats['group_stat'] = team_stats['group']+'_'+team_stats['stat']
    output = f'{TEAM_STATS_FOLDER}relational/{date}_team_season_stats.csv'
    relational_stats=transformEAV(team_stats,index=['season','teamId','domain','range','updateDate'],output=output)
    # rel_team_stats = pd.concat([rel_team_stats, relational_stats], ignore_index=True,output=output)


2025_season.csv


In [14]:
csv_files = glob.glob(os.path.join(TEAM_STATS_FOLDER,"game/*_team_game_stats.csv"))
for file in csv_files:
    filename = os.path.basename(file)
    print(filename)
    team_game_stats = pd.read_csv(file)
    date = filename[:10]
    team_game_stats['group_stat'] = team_game_stats['group']+'_'+team_game_stats['stat']
    output=f'{TEAM_STATS_FOLDER}relational/{date}_team_game_stats.csv'
    relational_stats = transformEAV(team_game_stats,index=['gamePk','gameDate','side','domain','range'],output=output)
    # rel_team_stats = pd.concat([rel_team_stats, relational_stats], ignore_index=True,output=output)

2025-07-13_team_game_stats.csv


In [12]:
combined_stats = []
#Combine player season stats
csv_files = glob.glob(os.path.join(PLAYER_STATS_FOLDER,'season/*_player_season_stats.csv'))
for file in csv_files:
    season_stats = pd.read_csv(file)
    filename = os.path.basename(file)
    print(filename)
    season = filename[:4]
    player_season_stats = pd.read_csv(file)
    player_season_stats['group_stat'] = player_season_stats['group']+'_'+player_season_stats['stat']
    output=f"{PLAYER_STATS_FOLDER}/relational/{season}_stats.csv"
    relational_stats = transformEAV(player_season_stats,index=['domain','season','playerId',"playerName","primaryPosition",'updateDate'])
    combined_stats.append(relational_stats)
output_df = pd.concat(combined_stats,ignore_index=True)
output_df.to_csv(output,index=False)

2025_player_season_stats.csv


In [10]:
combined_stats = []
#Add player game stats, one row per player per game
csv_files = glob.glob(os.path.join(PLAYER_STATS_FOLDER, 'game/*_player_game_stats.csv'))
for file in csv_files:
    filename = os.path.basename(file)
    print(filename)
    game_stats = pd.read_csv(file)
    date = filename[:10]
    game_stats['group_stat'] = game_stats['group']+'_'+game_stats['stat']
    output=f"{PLAYER_STATS_FOLDER}relational/{date}_game_stats.csv"
    relational_stats = transformEAV(game_stats, index = ["gamePk","side","playerId","positionsPlayed"])
    combined_stats.append(relational_stats)
output_df = pd.concat(combined_stats,ignore_index=True)
output_df.to_csv(output,index=False)

2025-07-13_player_game_stats.csv
